# 선형 회귀식의 계수를 찾는 법 - OLS VS. SGD
- 보스턴 집값 데이터 활용(RM VS Price)

### 필요한 모듈 import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 필요한 라이브러리 import 


### 데이터 수집 및 분할

In [3]:
from sklearn import datasets

boston = datasets.fetch_openml('boston')

c:\ProgramData\Anaconda3\envs\main\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
c:\ProgramData\Anaconda3\envs\main\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
x = np.array(boston.data.RM).reshape(-1,1)
y = boston.target

# 데이터 분할
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=1)

# 1. LinearRegression 모델을 사용한 경우

In [6]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(x_train,y_train)

print(reg.coef_,reg.intercept_)

y_pred = reg.predict(x_test)

from eval_score_function import eval_score

eval_score(y_test,y_pred)

[8.46109164] -30.571032410898336
mse = 36.517
rmse = 6.043
r2 = 0.602


(36.517214730838624, 6.042947520112898, 0.6015774471545623)

# 2. SGDRegressor with hyperparameter

In [53]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(max_iter=1000000000,eta0=0.0001, learning_rate ='invscaling',
                   loss='squared_error',
                   random_state=42)

reg.fit(x_train,y_train)

print(reg.coef_[0],reg.intercept_[0])

3.5471540937480923 0.497708365257496


In [54]:
y_pred = reg.predict(x_test)

eval_score(y_test,y_pred)

mse = 59.484
rmse = 7.713
r2 = 0.351


(59.484466468989275, 7.712617355281492, 0.3509923152707195)

# 3. SGDRegressor with scaling

In [49]:
# 표준화 스케일링을 사용하여 경사하강법 모델링

# 스케일링 데이터 준비
train_mean = np.mean(x_train,axis=0)
train_std = np.std(x_train, axis=0)

x_train_scaled = (x_train - train_mean) / train_std

x_test_scaled = (x_test - train_mean) / train_std

# 모델 객체 생성
reg = SGDRegressor(max_iter=1000000000,eta0=0.0001, learning_rate ='invscaling',
                   loss='squared_error',
                   random_state=42)

reg.fit(x_train_scaled,y_train)

# 계수 확인
print(reg.coef_,reg.intercept_)

# 평가지표
y_pred = reg.predict(x_test_scaled)

eval_score(y_test,y_pred)

[5.62395551] [21.47086473]
mse = 37.668
rmse = 6.137
r2 = 0.589


(37.66829778653543, 6.13745043047481, 0.5890185087752737)

# 4. SGDRegressor with StandardScaler()

In [62]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_train_scaled

# x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# 5. Pipeline with StandardScaler,LinearRegression or SGDRegrssor

In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

reg = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=1000000, eta0=0.01,\
                                 tol=0.0001, random_state=42, loss='squared_error'))
reg.fit(x_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg[1].coef_, reg[1].intercept_)

#회귀식 - pipeline()을 사용했기 때문에 SGDRegressor의 parameter가 reg객체의 1번 인덱스에 들어감
print("y = {:2f}X + {:.3f}".format(reg[1].coef_[0], reg[1].intercept_[0]))

# 예측 수행
y_pred = reg.predict(x_test)
from sklearn.metrics import mean_squared_error, r2_score
# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[5.84750366] [22.31897879]
y = 5.847504X + 22.319
MSE: 36.523
RMSE:  6.043
R2:  0.602
